<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
#!pip install imbalanced-learn -q
#!pip install swifter - q

#!pip install lightgbm
#!pip install xgboost

In [2]:
import pandas as pd
from tqdm import notebook
import re
from time import time 
#import nltk
import numpy as np
from numpy.random import default_rng
import transformers
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

from __future__ import unicode_literals, print_function
from spacy.lang.en import English

In [3]:
from imblearn.under_sampling import RandomUnderSampler

In [4]:
import torch
import transformers

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
import swifter

In [7]:
#import spacy
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#from spacy.pipeline import Lemmatizer
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

In [8]:
#tokenizer = transformers.BertTokenizer(
#    vocab_file='datasets/ds_bert/vocab.txt')

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
nlp = English()
nlp.add_pipe('sentencizer')

In [11]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [12]:
tokenizer.encode('He ran. He fell.')

[101, 2002, 2743, 1012, 2002, 3062, 1012, 102]

In [13]:
tokenizer.added_tokens_encoder

{'[PAD]': 0, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102, '[MASK]': 103}

In [14]:
for enc in tokenizer.encode('He ran. He fell. Aaaaaaaaaaa...'):
    display((enc,tokenizer.decode([f'{enc}'])))

(101, '[CLS]')

(2002, 'he')

(2743, 'ran')

(1012, '.')

(2002, 'he')

...

(11057, '##aa')

(1012, '.')

(1012, '.')

(1012, '.')

(102, '[SEP]')

In [15]:
RANDOM_STATE = 88

## Подготовка

In [16]:
start = time()
#df['text'].apply(lambda x: len(str(x).split()))
end = time()
end-start

0.0

In [17]:
url = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

In [18]:
df = pd.read_csv(url)

In [19]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [21]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [22]:
df['text'].duplicated().sum()

0

In [23]:
def clean_text(string):
    string = re.sub(r'[^a-z]', ' ', string, flags=re.IGNORECASE)
    string = string.split()
    string = ' '.join(string)
    return string.lower()

In [24]:
df['clean_text'] = df['text'].apply(lambda x:clean_text(x))

In [25]:
df['clean_text'].duplicated().sum()

1323

In [26]:
df.drop_duplicates(subset='clean_text', inplace=True)

In [27]:
df.reset_index(drop=True, inplace=True)

In [28]:
df['text'].str.lower().duplicated().sum()

0

In [29]:
df.isna().sum()

text          0
toxic         0
clean_text    0
dtype: int64

In [30]:
def sents(string):
    doc = nlp(string)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [31]:
df['word_count'] = df['text'].apply(lambda x: x.count(' ') + 1)

In [32]:
#df['sent_count'] = df['text'].apply(lambda x: len(sents(x)))

In [33]:
df['len'] = df['text'].apply(lambda x: len(x))

In [34]:
df['text'][154485]

'Turret #4\n\nWhy did the builder raise turret #4 on a high mount, while #3 sits on a normal low mount? It can only fire broadside, so why bother with added top weight? East of Borschov'

In [35]:
#df['text'][112719]

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157969 entries, 0 to 157968
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        157969 non-null  object
 1   toxic       157969 non-null  int64 
 2   clean_text  157969 non-null  object
 3   word_count  157969 non-null  int64 
 4   len         157969 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 6.0+ MB


In [37]:
#df['text_lower'] = df['mid_text'].str.lower()
#df.drop_duplicates(subset='text_lower', inplace=True)
#df.reset_index(drop=True, inplace=True)

In [38]:
def distribution_plt(df_list, name, val_list, plt_xlim=1):

    x_max = min([series.max() for series in df_list])* plt_xlim

    fig, ax = plt.subplots(2,1,figsize=(10,8))
    ax[0].boxplot([df for df in df_list], vert=False)

    ax[0].set_yticklabels(val_list)
    ax[0].set_xlabel(name)
    ax[0].set_xlim([0, x_max])

    ax[1].hist([df for df in df_list], histtype='stepfilled', bins=500)
    ax[1].set_ylabel('частота')
    ax[1].set_xlabel(name)
    ax[1].set_xlim([0, x_max])

    
    plt.suptitle(f'Ящики с усами и гистограмма для признака "{name}"')
    plt.legend(val_list);
    #plt.xlim((0,x_max))

In [39]:
name_dict = {'word_count' : 'количество слов',
 'sent_count' : 'количество предложений',
 'len' : 'количество символов'}

for col in ['word_count', 'sent_count', 'len']:
    
    distribution_plt(
        
        [df[df['toxic']==1][col], df[df['toxic']==0][col]],
        
        name_dict[col],
        
        ['токсичные\nкомментарии', 'не токсичные\nкомментарии'],
        
        0.1)

In [40]:
def word_rebuild(word_list):
    
    for word in range(len(word_list)):
        if len(word_list[word])>100:
            word_list[word] = ''.join(list(dict.fromkeys(word_list[word])))
    
    return word_list

In [ ]:
config = transformers.BertConfig.from_dict({
      "attention_probs_dropout_prob": 0.1,
      "directionality": "bidi",
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "hidden_size": 768,
      "initializer_range": 0.02,
      "intermediate_size": 3072,
      #"layer_norm_eps": 7e-2,
      "layer_norm_eps": 1e-12,
      #"layer_norm_eps": 4e-5,
      "max_position_embeddings": 512,
      "model_type": "bert",
      "num_attention_heads": 12,
      "num_hidden_layers": 12,
      "pad_token_id": 0,
      "pooler_fc_size": 768,
      "pooler_num_attention_heads": 12,
      "pooler_num_fc_layers": 3,
      "pooler_size_per_head": 128,
      "pooler_type": "first_token_transform",
      #"position_embedding_type": "absolute",
      #"transformers_version": "4.12.5",
      "use_cache":True,
      "type_vocab_size": 2,
      "vocab_size": 30522
}
)

In [42]:
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 4e-05,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.50.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

def chunker(string, max_tokens):

    sents_ = sents(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    
    if len(toks)>1:
        result = sum(toks,[])    
    else:
        result = toks[0]
    
    if len(result)>max_tokens:
        
        mid_sent = len(toks)//2
        
        if mid_sent == 1:
            
            if len(toks[1])<max_tokens:
                result = toks[1]    
            else:
                result = toks[0]
                
        elif mid_sent > 1:
            
            for s in range(1,mid_sent):

                mid_sents = toks[(mid_sent-s):mid_sent] + toks[mid_sent:(mid_sent+s)]#+toks[-1]
                mid_sents_s = sum(mid_sents,[])
                if len(mid_sents_s) > max_tokens:
                    break
                    result = sum(mid_sents[:-1],[])
        
        else:
            result = sum(toks,[])
    #return result
    return result[:max_tokens][:-1]+[102]

def chunker(string, max_tokens):

    max_tokens = max_tokens-1
    sents_ = sents(string)

    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=False) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    

    result = sum(toks,[])    
    
    if len(result)>max_tokens:
        if len(sents_)>2:
            result = toks[0]+toks[-1]
    
            
    return [101] + result[:max_tokens-2] + [102]

In [ ]:
def chunker(string, max_tokens=512):
    sents_ = sents(string)
    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=False) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
        
    result = sum(toks, [])#[:-1]
    return result 

In [623]:
def chunker(string, max_tokens=512):

    tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
    
    tokenized = tokenizer.encode(string, add_special_tokens=True)

    result = tokenized[:max_tokens]

    return result

def chunker(string, max_tokens=512):

    sents_ = sents(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    result = sum(toks, [])#[:-1]
    
    
    
    if len(set(result))>len(result)*0.7:
  
        result = list(dict.fromkeys(result))[:-1]
    
    
    result = result[:max_tokens-1]
    
    if result[-1]!=102:
        result = result[:max_tokens-1] + [102]

    
    
    return result

In [44]:
df['text'][4704]

"Dab said:\n\nMy impression remainst that the only reason this article is so long and detailed is the attempt to disguise from the casual reader the fact that the entire thing can be shrugged off in a one-liner.\n\nYes, that's probably true, but what is the alternative?  Many of us have been battling to make the CMT clear, but every time a statement is made on how crazy the theory is, the other side asks for a proof/citation.  And since that has happened almost every week for the last six months (and probably longer), the article has grown in size.  I'm not sure how we can overcome this problem, but I'm open to suggestions."

In [45]:
tokenizer.decode(chunker(df['text'][4704]))

"[CLS] dab said : my impression remainst that the only reason this article is so long and detailed is the attempt to disguise from the casual reader the fact that the entire thing can be shrugged off in a one - liner. yes, that's probably true, but what is the alternative? many of us have been battling to make the cmt clear, but every time a statement is made on how crazy the theory is, the other side asks for a proof / citation. and since that has happened almost every week for the last six months ( and probably longer ), the article has grown in size. i'm not sure how we can overcome this problem, but i'm open to suggestions. [SEP]"

In [121]:
tokenizer.decode(tokenizer.encode(df['text'][4704]))

"[CLS] dab said : my impression remainst that the only reason this article is so long and detailed is the attempt to disguise from the casual reader the fact that the entire thing can be shrugged off in a one - liner. yes, that's probably true, but what is the alternative? many of us have been battling to make the cmt clear, but every time a statement is made on how crazy the theory is, the other side asks for a proof / citation. and since that has happened almost every week for the last six months ( and probably longer ), the article has grown in size. i'm not sure how we can overcome this problem, but i'm open to suggestions. [SEP]"

In [46]:
df[df['text'].str.contains('I ass. I ass.')]['text']

4700    do go fuck off bastard\nDo Yyou Have a life?\n...
Name: text, dtype: object

In [52]:
tokenizer.decode(chunker(df['text'][4700], 512))

'[CLS] do go fuck off bastard do yyou have a life? go fuck off bastard and yank your cock through your ass. i hate you and hope you go away forever. lame is you fuck your mom. die die die and all that crap. this is for mahy mahonerz ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass. i ass.

In [137]:
#df[df['text'].apply(lambda x: len(x))>=4900]['text'].apply(lambda x: len(chunker(x))).describe()

In [47]:
tokenizer.decode(chunker(df['text'][82496]))

'[CLS] : file : jamaican railroad with railcar and station 1960. jpg hi, i found the above photo but i have no idea what railcar is shown where on jamaica. maybe you can help? cheers [SEP]'

In [53]:
df['text'].sample()

19151    Why?\n\nWhy would you talk about me like that,...
Name: text, dtype: object

In [54]:
df.sample(2)['text'].apply(lambda x: chunker(x,10)).iloc[0]

[101, 5432, 3718, 1012, 102, 4067, 2017, 2005, 2025, 102]

In [55]:
#df['text'][82496]

In [56]:
#df['text'][116571]

In [139]:
#

def chunker(string, max_tokens=512):


    
    tokenized = tokenizer.encode(string, add_special_tokens=False)
    len_wo_tokens = max_tokens-2
    
    if len(tokenized)> len_wo_tokens:
        
        result = tokenized[:(len_wo_tokens//2)] + tokenized[::-1][:(len_wo_tokens//2)][::-1]
    else:
        result = tokenized

    return [101] + result + [102]

In [140]:
#chunker(df['text'][0])

In [141]:
#OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
#Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [142]:
#max_batch = 512

In [143]:
#init_config = config

In [144]:
#batch_size = 512

tokenized = X_train.swifter.apply(lambda x: chunker(x,max_batch))
#tokenized = text.apply(lambda x: chunker(x,max_batch))

max_len = 0

for tok in tokenized.values:
    if len(tok) > max_len:
        max_len = len(tok)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)


model = transformers.BertModel.from_pretrained('bert-base-uncased', config=init_config)
#model.config.max_position_embeddings = max_batch


batch_size = batch_size

embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

train_features = np.concatenate(embeddings)

test_tokenized = X_test.swifter.apply(lambda x: chunker(x,max_batch))


max_len = 0
for tok in test_tokenized.values:
    if len(tok) > max_len:
        max_len = len(tok)

padded = np.array([i + [0]*(max_len - len(i)) for i in test_tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

test_embeddings = []


for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            test_batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        test_embeddings.append(test_batch_embeddings[0][:,0,:].numpy())

test_features = np.concatenate(test_embeddings)

In [145]:
#train_features.shape

In [146]:
#test_features.shape

In [624]:
class bert_emb(object):


    def __init__(self, data, init_config,  batch_size=32, max_batch=512):

        self.data = data
        self.batch_size = batch_size
        self.max_batch = max_batch
        self.init_config = init_config 

        #self.config.max_position_embeddings = max_batch
        self.model = transformers.BertModel.from_pretrained('bert-base-uncased', config=init_config)
    
    def embedder(self):

        tokenized = self.data.swifter.apply(lambda x: chunker(x, self.max_batch))
        #tokenized = self.data.apply(lambda x:list(tokenizer(x,add_special_tokens=True)))#.iloc[:self.max_batch]
        #tokenized = self.data.apply(lambda x: tokenizer(x,add_special_tokens=True))
        #tokenized = tokenized.values.tolist()
        
        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)
    
        #padded = [i + [0]*(max_len - len(i)) for i in tokenized.values]
        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])#,dtype=np.int64)
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(len(padded) // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features
        

In [148]:
#bert_emb(df['text'][0:5],config,1).embedder().shape

In [884]:
def logy(feats, target):
    lr = LogisticRegression(random_state=RANDOM_STATE,max_iter=feats.shape[1],class_weight='balanced')
    lr.fit(feats, target)
    scores = cross_val_score(lr, feats, target, cv=5, scoring='f1_macro')
    return scores

In [521]:
y_train.value_counts(normalize=1)

toxic
0    0.906738
1    0.093262
Name: proportion, dtype: float64

In [56]:
#df[df['len']<=df['len'].quantile(0.9)].shape

In [57]:
# = df.sample(frac=0.5, random_state=RANDOM_STATE)

In [58]:
#df_test = df[df.index.isin(df_train.index)]

In [59]:
#df_train.shape,df_test.shape

In [60]:
#len_q_upper = df_train['len'].quantile(0.95)

In [61]:
#len_q_lower = df_train['len'].quantile(0.05)

In [62]:
#df_train.shape

In [63]:
#df_train = df_train[(df_train['len'] >= len_q_lower) & (df_train['len'] <= len_q_upper)]

df_for_test1 = df[df['toxic']==1].sample(n=512,random_state=RANDOM_STATE)

df_for_test = df[df['toxic']==0].sample(n=512,random_state=RANDOM_STATE)

df_for_test = pd.concat([df_for_test,df_for_test1])

X_train=df_for_test['text']
y_train = df_for_test['toxic']

X_test = df[~df.index.isin(X_test.index)].sample(n=1024,random_state=RANDOM_STATE)

y_test = X_test['toxic']
X_test = X_test['text']

In [64]:
#bert_emb(df['text'][0:1],config,batch_size=1,max_batch=10).embedder()[0]==bert_emb(df['text'][0:5],config,batch_size=1,max_batch=10).embedder()[0]

In [65]:
1024*32

32768

In [891]:
1024*16

16384

In [847]:
1024*8

8192

In [67]:
#best - 32768

In [68]:
df.shape

(157969, 5)

In [885]:
samp = df.sample(n=16384, random_state=RANDOM_STATE)

In [781]:
#for i in samp.iloc[:,2:-1]:
#    display(i, samp[i].describe())

X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['toxic'],
    test_size = 0.50,
    random_state = RANDOM_STATE,
    stratify = df['toxic'])

In [886]:
X_train, X_test, y_train, y_test = train_test_split(
    samp['text'],
    samp['toxic'],
    test_size = 0.50,
    random_state = RANDOM_STATE,
    stratify = samp['toxic'])

X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['toxic'],
    test_size = 0.50,
    random_state = RANDOM_STATE,
    stratify = df['toxic'])

In [887]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8192,), (8192,), (8192,), (8192,))

In [893]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train))

In [894]:
X_resample.shape, y_resample.shape

((1580, 1), (1580, 1))

In [895]:
X_resample.head(), y_resample.head()

(                                                    text
 78488  "\n\nAd Tag Placed - This Article Needs MAJOR ...
 51529  ", 22 November 2008 (UTC)\nSJW is of remarkabl...
 98437  And there's no real difference between a mini-...
 72013                               Also, 128.61.126.234
 40034  File:उलझे अनसुलझे मसलों को मिनटों में सुलझा दू...,
        toxic
 78488      0
 51529      0
 98437      0
 72013      0
 40034      0)

In [896]:
X_resample.tail(), y_resample.tail()

(                                                    text
 66205  As i said before i misread 2000 for 2008 which...
 45327  CSS Naked Day \n\nYOU ARE A FUCKING DUCHEBAG!!...
 93465  Maybe it'd be more encyclopedic if you got fuc...
 10859  Warning an editor that you've pulled this crap...
 94829                 R u gay? \n\nHi, are you a faggot?,
        toxic
 66205      1
 45327      1
 93465      1
 10859      1
 94829      1)

In [788]:
X_resample.shape

(382, 1)

In [789]:
config = transformers.BertConfig.from_dict({
      "attention_probs_dropout_prob": 0.1,
      "directionality": "bidi",
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "hidden_size": 768,
      "initializer_range": 0.02,
      "intermediate_size": 3072,
      "layer_norm_eps": 1e-12,
      "max_position_embeddings": 512,
      "model_type": "bert",
      "num_attention_heads": 12,
      "num_hidden_layers": 12,
      "pad_token_id": 0,
      "pooler_fc_size": 768,
      "pooler_num_attention_heads": 12,
      "pooler_num_fc_layers": 3,
      "pooler_size_per_head": 128,
      "pooler_type": "first_token_transform",
      "type_vocab_size": 2,
      "vocab_size": 30522
}
)

In [790]:
X_resample.shape[0]

382

X_resample = X_resample['text'].sort_index()
y_resample = y_resample['toxic'].sort_index()

In [791]:
X_resample.head(), X_resample.tail()

(                                                     text
 69568   "\n\n Mathematics \n\n""In 1637, increasing su...
 120258  If there is no issue with the language then wh...
 42464   I have backed them up all over the place. I re...
 29489   No, the hyphen does quite the reverse, and it'...
 91150   "\n\nI don't know what you want to do with thi...,
                                                      text
 93047       Hey\nYou're a fucking cunt. Go kill yourself.
 143085  Wesley Bartz likes to suck cock. Loves that co...
 26043   "\n\nKeep on playing cat and mouse for as long...
 108751  I am a power hungry administrator! Who likes t...
 72595   Suck it dick face from IP Address: 71.195.241....)

In [792]:
y_resample.head(), y_resample.tail()

(        toxic
 69568       0
 120258      0
 42464       0
 29489       0
 91150       0,
         toxic
 93047       1
 143085      1
 26043       1
 108751      1
 72595       1)

In [855]:
X_resample.shape

(778, 1)

In [794]:
778/2

389.0

In [857]:
class BertTokenizer(object):

    def __init__(self, text=[]):
        self.text = text

        # For DistilBERT:
        self.model_class, self.tokenizer_class, self.pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

        # Load pretrained model/tokenizer
        self.tokenizer = self.tokenizer_class.from_pretrained(self.pretrained_weights)

        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    def get(self):

        df = pd.DataFrame(data={"text":self.text})
        tokenized = df["text"].swifter.apply((lambda x: self.tokenizer.encode(x, add_special_tokens=True)[:512]))

        max_len = 0
        for i in tokenized.values:
            if len(i) > max_len:
                max_len = len(i)

        padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded)
        attention_mask = torch.tensor(attention_mask)

        with torch.no_grad(): last_hidden_states = self.model(input_ids, attention_mask=attention_mask)
        
        features = last_hidden_states[0][:, 0, :].numpy()

        return features

In [859]:
class BertTokenizer(object):

    def __init__(self, text=[], batch_size=32):
        self.text = text
        self.batch_size = batch_size
        # For DistilBERT:
        self.model_class, self.tokenizer_class, self.pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

        # Load pretrained model/tokenizer
        self.tokenizer = self.tokenizer_class.from_pretrained(self.pretrained_weights)

        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    def get(self):

        df = pd.DataFrame(data={"text":self.text})
        tokenized = df["text"].swifter.apply((lambda x: self.tokenizer.encode(x, add_special_tokens=True)[:512]))

        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)
    
        #padded = [i + [0]*(max_len - len(i)) for i in tokenized.values]
        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])#,dtype=np.int64)
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(len(padded) // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features

In [899]:
X_resample.shape[0]/2

790.0

In [900]:
dembs = BertTokenizer(text=X_resample['text'], batch_size=790).get()

Pandas Apply:   0%|          | 0/1580 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [902]:
logy(dembs, y_resample['toxic']).mean()

0.8721352638112192

In [903]:
np.array([0.85350318, 0.8625    , 0.87179487, 0.82352941, 0.84146341]).mean()

0.850558174

In [907]:
X_test.head(), y_test.head()

(65906     Question\n\nAnybody have any idea how to find ...
 14415     Completely agree with Zouavman on this one, Nu...
 101111    Press Tilton\nThis section needs more and bett...
 44073     Eyéghé \n\nThe article spells it Eyéghé, Eyégh...
 74793     Fuck off, before you give me the 'wikipedia is...
 Name: text, dtype: object,
 65906     0
 14415     0
 101111    0
 44073     0
 74793     1
 Name: toxic, dtype: int64)

In [918]:
test_sample = pd.DataFrame(X_test).join(pd.DataFrame(y_test)).sample(n=dembs.shape[0], random_state=RANDOM_STATE)

In [919]:
test_sample.head()

,text,toxic
87233,That's not the point. I had made the claim tha...,0
114044,It is just as much garbage to claim that the y...,1
42621,"""\n\nObjection: You appear to ignore the follo...",0
30033,"Srnec, you blunt idiot! I wrote that article, ...",1
100784,IPA? \n\nThe usage of IPA is very confusing. /...,0


In [ ]:
#array([0.88311688, 0.88303797, 0.76315789, 0.82868042, 0.82820379])

In [921]:
dembs_test = BertTokenizer(text=test_sample['text'], batch_size=790).get()

Pandas Apply:   0%|          | 0/1580 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [584]:
y_train.value_counts(normalize=2)

toxic
0    0.906738
1    0.093262
Name: proportion, dtype: float64

In [731]:
def f1_eval(y_true, y_pred,):
    #y_true = dtrain.get_label()
    #err = 1 - f1_score(grid_pred, y_test, average=None)[1]
    err = f1_score(y_pred, y_true, average=None)[1]
    return  err

In [754]:
def f1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = f1_score(y_true, y_pred, average=None)[1]

    return 'err', err

In [732]:
from sklearn.metrics import make_scorer
my_func = make_scorer(f1_eval, greater_is_better=True)

In [834]:
param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        'models__C': [0.1, 1]#,
        #'models__penalty': ['l1', 'l2']
          
    },

    {
        'models': [SVC(random_state=RANDOM_STATE, class_weight='balanced')],
        'models__degree': range(3,8),
        'models__kernel': ['poly','rbf','sigmoid']
    },
    {
        'models': [XGBClassifier(random_state=RANDOM_STATE, maximize=True)],
        'models__n_estimators':[500],
        'models__learning_rate':[0.1, 0.07, 0.05],
        'models__eval_metric': [f1_xgb]
    }
    #{
    #    'models': [LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
    #    'models__n_estimators': [1000],
    #    'models__learning_rate': [0.1, 0.01, 0.03, 0.05, 0.07]
    #}
]

In [832]:
param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        'models__C': [0.1, 1]#,
        #'models__penalty': ['l1', 'l2']
          
    }
]

In [904]:
grid_search = GridSearchCV(
    pipe, 
    param_grid, 
    cv=5,
    #scoring='f1_macro',
    scoring=my_func,
    n_jobs=-1
)
#grid_search.fit(X_resample[:X_test.shape[0]], y_resample['toxic'][:X_test.shape[0]])

#grid_search.fit(embs, y_resample['toxic'].iloc[:test_sample.shape[0]])
#grid_search.fit(embs, y_resample)
#grid_search.fit(embs, y_train)
grid_search.fit(dembs, y_resample['toxic'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('models',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=88))]),
             n_jobs=-1,
             param_grid=[{'models': [LogisticRegression(class_weight='balanced',
                                                        random_state=88)],
                          'models__C': [0.1, 1],
                          'models__solver': ['newton-cg', 'liblinear']},
                         {'models': [SVC(class_weight='balanced',
                                         random_state=88)],
                          'models__degree': range(3, 8),...
                                                   max_delta_step=None,
                                                   max_depth=None,
                                                   max_leaves=None,
                                                   maximize=True,
                                                   min_child_weight=None,
                                                   missing=nan,
                                                   monotone_constraints=None,
                                                   multi_strategy=None,
                                                   n_estimators=None,
                                                   n_jobs=None, ...)],
                          'models__eval_metric': [<function f1_xgb at 0x00000239B0724180>],
                          'models__learning_rate': [0.1, 0.07, 0.05],
                          'models__n_estimators': [500]}],
             scoring=make_scorer(f1_eval, response_method='predict'))

In [905]:
pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'mean_test_score','params']].sort_values('rank_test_score')#.loc[3]['params']#.head(16)

,rank_test_score,mean_test_score,params
1,1,0.880082,{'models': LogisticRegression(class_weight='ba...
0,2,0.878817,{'models': LogisticRegression(class_weight='ba...
4,3,0.876460,"{'models': SVC(class_weight='balanced', random..."
7,4,0.875847,"{'models': SVC(class_weight='balanced', random..."
5,5,0.875042,"{'models': SVC(class_weight='balanced', random..."
17,5,0.875042,"{'models': SVC(class_weight='balanced', random..."
8,5,0.875042,"{'models': SVC(class_weight='balanced', random..."
14,5,0.875042,"{'models': SVC(class_weight='balanced', random..."
11,5,0.875042,"{'models': SVC(class_weight='balanced', random..."
3,10,0.872716,{'models': LogisticRegression(class_weight='ba...


In [758]:
#grid_search.best_estimator_

In [922]:
grid_pred = grid_search.best_estimator_.predict(dembs_test)#[:X_train.shape[0]])

In [923]:
y_test.shape

(8192,)

In [924]:
dembs_test.shape

(1580, 768)

In [925]:
grid_pred.shape

(1580,)

In [842]:
#0.6818181818181818

In [843]:
#y_test[test_sample.index]

In [844]:
#0.5166340508806262
#0.670076726342711

In [926]:
f1_score(grid_pred, test_sample['toxic'])

0.5977011494252874

In [927]:
print(classification_report(grid_pred, test_sample['toxic']))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      1292
           1       0.88      0.45      0.60       288

    accuracy                           0.89      1580
   macro avg       0.89      0.72      0.77      1580
weighted avg       0.89      0.89      0.87      1580



In [748]:
res =pd.DataFrame({'a':grid_pred, 'b':y_test})

In [749]:
res.shape

(2048, 2)

In [750]:
res[res['a']!=res['b']]['b'].value_counts()

b
0    123
1     49
Name: count, dtype: int64

## Обучение

In [ ]:
#from phik.report import plot_correlation_matrix
#from phik import phik_matrix

In [ ]:
#df.info()

phik_overview = phik_matrix(df[['toxic','word_count']],verbose=False)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Blues',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(18, 14)
) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_lemm'], 
    df['toxic'], 
    random_state=RANDOM_STATE,
    stratify=df['toxic']
)

In [ ]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 


In [ ]:
y_resample.value_counts()

In [ ]:
pd.DataFrame(y_test).value_counts()

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны